In [ ]:
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn import svm, metrics

In [1]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
    
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values

	if dropnan:
		agg.dropna(inplace=True)
	return agg
 

In [5]:
# load dataset
dataset = read_csv('/content/drive/MyDrive/Colab Notebooks/kumdan_2019v1.csv', header=0, index_col=0)
values = dataset.values

#integer encode direction
#encoder = LabelEncoder()
#values[:,4] = encoder.fit_transform(values[:,4])

# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)


# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
#reframed = series_to_supervised(values, 1, 1)

# drop columns we don't want to predict
reframed.drop(reframed.columns[[11,12,13,14,15,16,17,18,19,19]], axis=1, inplace=True)
print(reframed.head())
 


SyntaxError: ignored

In [ ]:
# split into train and test sets
values = reframed.values
n_train_hours = 346 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
print(train_X.shape[1], train_X.shape[2])


In [ ]:
# design network
model = Sequential()
#model.add(LSTM(75, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LSTM(75, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LSTM(25))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='observation')
pyplot.plot(history.history['val_loss'], label='prediction')
pyplot.legend()
pyplot.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))


In [ ]:
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

In [ ]:
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

In [ ]:
#print(inv_y[0], inv_yhat[0])
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.5f' % rmse)

In [ ]:
# calculate MAPE
mape = np.mean(np.abs((inv_y - inv_yhat) / inv_y))
print('Test MAPE: %.5f' % mape)

In [ ]:
# calculate index of agreement
ioa= 1 -(np.sum((inv_y-inv_yhat)**2))/(np.sum((np.abs(inv_yhat-np.mean(inv_y))+np.abs(inv_y-np.mean(inv_y)))**2))
print('Test IOA: %.5f' % ioa)

In [ ]:
scores = model.evaluate(train_X, train_y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))